In [1]:
import pandas as pd
import os




In [2]:
def get_role_averages():
    frames = []

    zones = ['Center(C)',
     'Left Side Center(LC)',
     'Left Side(L)',
     'Right Side Center(RC)',
     'Right Side(R)',
     'Center(C)',
     'Left Side Center(LC)',
     'Left Side(L)',
     'Right Side Center(RC)',
     'Right Side(R)',
     'Center(C)',
     'Left Side(L)',
     'Right Side(R)',
     'Back Court(BC)',
     'Center(C)']

    ranges = ['16-24 ft.',
     '16-24 ft.',
     '16-24 ft.',
     '16-24 ft.',
     '16-24 ft.',
     '24+ ft.',
     '24+ ft.',
     '24+ ft.',
     '24+ ft.',
     '24+ ft.',
     '8-16 ft.',
     '8-16 ft.',
     '8-16 ft.',
     'Back Court Shot',
     'Less Than 8 ft.']
    df = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/site_Data/refs/heads/master/lebron.csv')
    for year in range(2024,2026):

        year_df = df[df.year==year]


        roles = year_df['Offensive Archetype'].unique()
        for role in roles:

            role_avg=pd.DataFrame({'SHOT_ZONE':zones,'SHOT_ZONE_RANGE':ranges })
            role_avg['FGA'] = 0
            role_avg['FGM'] = 0

            role_df = year_df[year_df['Offensive Archetype']==role]

            for player_id in role_df['NBA ID'].unique():
                #print(player_id)
                if os.path.isfile(str(year)+'/'+str(int(player_id))+'.csv'):
                    playerdf = pd.read_csv(str(year)+'/'+str(int(player_id))+'.csv')
                    sum_df = playerdf.groupby(['SHOT_ZONE_RANGE','SHOT_ZONE_AREA','SHOT_MADE_FLAG']).size().unstack(fill_value=0).reset_index()

                    #sum_df.drop(columns=['SHOT_MADE_FLAG'],inplace=True)
                    if len(sum_df.columns) ==4:
                        sum_df.columns = ['SHOT_ZONE_RANGE','SHOT_ZONE_AREA','Missed','Made']
                    elif len(sum_df.columns) ==3:
                         sum_df.columns = ['SHOT_ZONE_RANGE','SHOT_ZONE_AREA','Missed']
                         sum_df['Made']=0
                    else:
                        sum_df.columns = ['SHOT_ZONE_RANGE','SHOT_ZONE_AREA']
                        sum_df['Missed']=0
                        sum_df['Made']=0

                    sum_df['FGM'] = sum_df['Made']
                    sum_df['FGA'] = sum_df['Made'] + sum_df['Missed']

                    sum_df.drop(columns=['Made','Missed'],inplace=True)
                    sum_df['role'] = role
                    sum_df['year']=year
                    frames.append(sum_df)

        print(year)
    return pd.concat(frames)
        
df = get_role_averages()

2024
2025


In [3]:
df

,SHOT_ZONE_RANGE,SHOT_ZONE_AREA,FGM,FGA,role,year
0,16-24 ft.,Center(C),16,37,Post Scorer,2024
1,16-24 ft.,Left Side Center(LC),12,32,Post Scorer,2024
2,16-24 ft.,Left Side(L),2,7,Post Scorer,2024
3,16-24 ft.,Right Side Center(RC),10,33,Post Scorer,2024
4,16-24 ft.,Right Side(R),2,4,Post Scorer,2024
...,...,...,...,...,...,...
6,24+ ft.,Right Side(R),1,3,Slasher,2025
7,8-16 ft.,Center(C),1,3,Slasher,2025
8,8-16 ft.,Left Side(L),1,1,Slasher,2025
9,8-16 ft.,Right Side(R),5,7,Slasher,2025


In [4]:
all_averages = df.groupby(['year','role','SHOT_ZONE_RANGE','SHOT_ZONE_AREA']).sum()[['FGM','FGA']].reset_index()
for year in df.year.unique():
    year_df= all_averages[all_averages.year==year].reset_index(drop=True)
    year_df.drop(columns='year',inplace=True)
    year_df.to_csv('roles/'+str(year)+'.csv',index = False)
    print(year_df)

                  role  SHOT_ZONE_RANGE         SHOT_ZONE_AREA   FGM   FGA
0    Athletic Finisher        16-24 ft.              Center(C)    64   191
1    Athletic Finisher        16-24 ft.   Left Side Center(LC)    52   172
2    Athletic Finisher        16-24 ft.           Left Side(L)    47   141
3    Athletic Finisher        16-24 ft.  Right Side Center(RC)    67   219
4    Athletic Finisher        16-24 ft.          Right Side(R)    49   123
..                 ...              ...                    ...   ...   ...
190      Versatile Big         8-16 ft.              Center(C)    89   210
191      Versatile Big         8-16 ft.           Left Side(L)    55   136
192      Versatile Big         8-16 ft.          Right Side(R)    59   140
193      Versatile Big  Back Court Shot         Back Court(BC)     0     4
194      Versatile Big  Less Than 8 ft.              Center(C)  1118  1805

[195 rows x 5 columns]
                  role  SHOT_ZONE_RANGE         SHOT_ZONE_AREA  FGM  FGA
0  

In [5]:
all_averages

,year,role,SHOT_ZONE_RANGE,SHOT_ZONE_AREA,FGM,FGA
0,2024,Athletic Finisher,16-24 ft.,Center(C),64,191
1,2024,Athletic Finisher,16-24 ft.,Left Side Center(LC),52,172
2,2024,Athletic Finisher,16-24 ft.,Left Side(L),47,141
3,2024,Athletic Finisher,16-24 ft.,Right Side Center(RC),67,219
4,2024,Athletic Finisher,16-24 ft.,Right Side(R),49,123
...,...,...,...,...,...,...
384,2025,Versatile Big,24+ ft.,Right Side(R),5,18
385,2025,Versatile Big,8-16 ft.,Center(C),42,67
386,2025,Versatile Big,8-16 ft.,Left Side(L),19,48
387,2025,Versatile Big,8-16 ft.,Right Side(R),12,24
